In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import warnings
warnings.filterwarnings("ignore")
from math import radians, cos, sin, asin, sqrt
from sklearn import linear_model
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

c:\users\sayeed alvi\appdata\local\programs\python\python37\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [5]:
data.shape
#Creating a new column Date from the column "start_time" of the dataframe 
data["start_time"] = pd.to_datetime(data["start_time"])

In [7]:
data = data[data['end_lat'] != 0]
data1.shape

(586779, 13)

In [83]:
#Creating a new column Date from the column "start_time" of the dataframe 
data["Date"] = pd.to_datetime(data["start_time"])


In [8]:
#Creating a copy of data since loading data takes a lot of time
data1 = data.copy()

In [ ]:
# import seaborn as sns
# # sns.countplot(data['passholder_type'], color='gray')
# ax = sns.countplot(x="passholder_type", data= data1)

In [10]:
#Dropping null values
data1 = data1.dropna() 
# Converting float values to interger
data1["end_station"] = data2["end_station"].astype(int)  
 #setting index to Date column
data1 = data2.sort_index() #Arranging dates 

In [84]:
# data1.head()

In [12]:
#length of end station is 142 and length of start station is 140
end_stn = list(set(data1.end_station))
start_stn = list(set(data1.start_station))
"""As length is not same of these two list, a search method is used to identify which element 
is missing from the start station, hence the next chunk of code is to find
the station number not in the start station list"""
remove_stn = []
for i in range(len(end_stn)):
    if end_stn[i] not in start_stn:
        remove_stn.append(end_stn[i])
#Updating end station list
end_stn = [stn for stn in end_stn if stn not in remove_stn]

In [13]:
#Removing rows with station number 4110,4143 and 4118 as it is discarded from the analysis
data1= data1[data1['end_station']!=4110]
data1 = data1[data1['end_station']!=4118]
data1 = data1[data1['start_station']!=4143]

In [14]:
#Setting plan_duration = 0 for passholder_type = Walk-up
data1['plan_duration'].loc[data1['passholder_type'] == "Walk-up"] = 0

#Setting annual pass = flex as both of them are the same
data1['passholder_type'].loc[data1['passholder_type'] == "Flex Pass"] = "Annual Pass"

In [15]:
#Calculating trip duration in minutes
data1['start_time']= pd.to_datetime(data1['start_time']) 
data1['end_time']= pd.to_datetime(data1['end_time']) 
data1['trip_duration_mins'] = (data1.end_time - data1.start_time)/ timedelta(minutes=1)

In [16]:
#Creating a copy of data since it takes time to load data
data2 = data1.copy()
data2 = data2[(data2['trip_duration_mins']>=0)]

In [85]:
print(data2.shape)
data2.head()

In [ ]:
#Create dummy variables for passholder type
one_hot_pass = pd.get_dummies(data3['passholder_type']).rename(columns={
            'Flex Pass': 'annual',
            'Monthly Pass': 'monthly',
            'One Day Pass': 'one_day',
            'Walk-up':'walk_up'})

#Create dummy variables for trip route category
one_hot_trip_type = pd.get_dummies(data3['trip_route_category']).rename(columns={
            'Round Trip': 'round_trip',
            'One Way': 'one_way'})

In [ ]:
data2 = pd.concat([data2, one_hot_trip_type,one_hot_pass], axis=1, sort=False)

In [ ]:
data2.to_csv('pricing_optimization.csv')

In [ ]:
import googlemaps

#Setting up my google api key
apiKey = 'XXXXXXXXXXXXXXXXXXXXXXXXXX'
gmaps = googlemaps.Client(key=apiKey)

#Entering latitude and longitude to be used
start_loc = "34.04739000000001,-118.21884900000035"
end_loc = "34.04652000000171,-118.23741100000254"

#Calculating distance by setting the travel mode = bicycling
result = gmaps.distance_matrix(start_loc, end_loc, mode="bicycling")

#Saving Distance and duration in the variables
dist = result["rows"][0]["elements"][0]["distance"]["value"]
dur = result["rows"][0]["elements"][0]["duration"]["value"]
dur = dur/60.0

#printing results
print(" Cycling Distance(meters): ", dist)
print(" Expected Cycling Time(min): ", dur)
print(result)

In [ ]:
unique_coordinates = data2[data2['round_trip']==0]
unique_coordinates = data2[['start_lat', 'start_lon', 'end_lat','end_lon']].drop_duplicates()
print(unique_coordinates.shape)

In [ ]:
def distance_calc(df):
    start_loc = (df['start_lat'], df['start_lon'])
    end_loc = (df['end_lat'], df['end_lon'])
    #Setting up my google api key
    apiKey = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
    gmaps = googlemaps.Client(key=apiKey)
    result = gmaps.distance_matrix(start_loc, end_loc, mode="bicycling")
    dist = result["rows"][0]["elements"][0]["distance"]["value"]
    return dist

In [ ]:
import googlemaps

#Dropping rows where coordinates are 0,0 
unique_coordinates = unique_coordinates.loc[(unique_coordinates!=0).any(axis=1)]

#Calculating distance
unique_coordinates['distance'] = unique_coordinates.head(5).apply (lambda row: distance_calc(row),axis=1)
unique_coordinates.head(10)

In [ ]:
#Calculating Distance traveled 
def calc_dist(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [ ]:
data2 = data2[(data2['trip_duration_mins']>=0) & (data2['trip_duration_mins']<50)]
data2 = data2[data2['Distance'] <50]
data2.shape

In [ ]:
month = 'month'
week = 'week'
day = 'day'
hour = 'hour'
def grouping(data,col):
    '''
    Stacking dataset on the basis of start_station
    Args: data = dataframe
          col = column name/ time frame
    Returns: Stacked dataframe with index resetted
    '''
    df = data.copy()
    df["Year"]=df.start_time.dt.year
    if col.lower() == 'month':
        col = col.capitalize()
        df[col] = df.start_time.dt.month
    elif col.lower() == 'week':
        col = col.capitalize()
        df[col] = df.start_time.dt.week
    elif col.lower() == 'day':
        col = col.capitalize()
        df[col] = df.start_time.dt.day
    elif col.lower() == 'hour':
        col = col.capitalize()
        df[col] = df.start_time.dt.hour
    else:
        return print('Unable to extract time from index!!')
    df_sum = df.groupby(by=["start_station","Year",col]).sum()[['Distance', 'trip_duration_mins']]
    df_count = df.groupby(by=["start_station","Year",col]).count()[['trip_id']]
    df = pd.concat([df_sum, df_count], axis=1)
    df = df.reset_index()
    return df

In [ ]:
data3 = grouping(data2,month)
#data3 = grouping(common,week)

In [ ]:
data3.head(20)

In [ ]:
#Setting offset for continuity in months
data3.loc[data3['Year']==2017, 'Month'] = data3['Month']+12
data3.loc[data3['Year']==2018, 'Month'] = data3['Month']+24

In [ ]:
data3 = data3[data3['start_station']!=4276]

In [ ]:
station = list(set(data3.start_station))
r_squared = []
#Looping over all the stations to fit linear regressions individually
for i in range(len(station)):
    stn = station[i]
    df = data3.loc[data3['start_station']==stn]
    x = df[['Distance','trip_duration_mins']]
    y = df["trip_id"]
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    print(X_train)
    reg = linear_model.LinearRegression()
    print(stn)
    est2 = reg.fit(X_train,y_train)
    ypred = est2.predict(x) #predicting y values on test dataset
    score = r2_score(y,ypred) #Calculating r-squared value on the test data
    r_squared.append(score)
r_squared

In [ ]:
df = data3.loc[data3['start_station']==4273]
x = df[['Distance','trip_duration_mins']]
y = df["trip_id"]
X_train, X_test, y_train, y_test = train_test_split( #Splitting data into train & test
            x, y, test_size=0.33, random_state=42)
reg = linear_model.LinearRegression()
est2 = reg.fit(X_train,y_train)
ypred = est2.predict(X_test) #predicting y values on test dataset
score = r2_score(y_test,ypred) #Calculating r-squared value on the test data
#r_squared.append(score)
score

In [ ]:
#Checking whether there is time series in the data
import matplotlib.pyplot as plt
plt.plot(df['Week'],df['trip_id'])
plt.show()